# Up and Running with TensoFlow
- Creating and running graphs

In [1]:
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.compat.v1.reset_default_graph()
    #tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "tensorflow"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv1D, MaxPool1D
from tensorflow.keras.optimizers import Adam
print(tf.__version__)

2.1.0


In [3]:
#Construction phase for building computation graphs
#Execution phase for running a loop that evaluates a training step


# Linear Regression 
## Using the Normal Equation
- eval is not supported when eager execution is enabled, is .numpy() what you're looking for?

In [5]:
from sklearn.datasets import fetch_california_housing
tf.compat.v1.disable_eager_execution()

reset_graph()

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.linalg.inv(tf.matmul(XT,X)),XT), y)

with tf.compat.v1.Session() as sess:
    theta_value = theta.eval()

In [6]:
theta_value

array([[-3.7225266e+01],
       [ 4.3568176e-01],
       [ 9.3872147e-03],
       [-1.0598953e-01],
       [ 6.3939309e-01],
       [-4.1104349e-06],
       [-3.7780963e-03],
       [-4.2437303e-01],
       [-4.3785891e-01]], dtype=float32)

In [7]:
# Compare with numpy

X = housing_data_plus_bias
y = housing.target.reshape(-1,1)
theta_numpy = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
print(theta_numpy)

[[-3.69419202e+01]
 [ 4.36693293e-01]
 [ 9.43577803e-03]
 [-1.07322041e-01]
 [ 6.45065694e-01]
 [-3.97638942e-06]
 [-3.78654265e-03]
 [-4.21314378e-01]
 [-4.34513755e-01]]


In [8]:
# Compare with Scikit-Learn

from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(housing.data, housing.target.reshape(-1,1))

print(np.r_[lin_reg.intercept_.reshape(-1,1), lin_reg.coef_.T])

[[-3.69419202e+01]
 [ 4.36693293e-01]
 [ 9.43577803e-03]
 [-1.07322041e-01]
 [ 6.45065694e-01]
 [-3.97638942e-06]
 [-3.78654265e-03]
 [-4.21314378e-01]
 [-4.34513755e-01]]


# Using Batch Gradient Descent
- First scale the feature vectors

In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m,1)), scaled_housing_data]

In [10]:
print(scaled_housing_data_plus_bias.mean(axis = 0))
print(scaled_housing_data_plus_bias.mean(axis = 1))
print(scaled_housing_data_plus_bias.mean())
print(scaled_housing_data_plus_bias.shape)

[ 1.00000000e+00  6.60969987e-17  5.50808322e-18  6.60969987e-17
 -1.06030602e-16 -1.10161664e-17  3.44255201e-18 -1.07958431e-15
 -8.52651283e-15]
[ 0.38915536  0.36424355  0.5116157  ... -0.06612179 -0.06360587
  0.01359031]
0.11111111111111005
(20640, 9)


# Manually computing the gradients

In [11]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='Y')

#Contain random values given its shape and range values
theta = tf.Variable(tf.random.uniform([n+1,1], -1.0,1.0, seed=42), name='theta')

y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y

mse = tf.reduce_mean(tf.square(error), name='MSE')
gradients = 2/m * tf.matmul(tf.transpose(X), error)

#Assigns new value to a variable
training_op = tf.compat.v1.assign(theta, theta - learning_rate * gradients)

init = tf.compat.v1.global_variables_initializer()

with tf.compat.v1.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE = ", mse.eval())
            
        sess.run(training_op)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Epoch 0 MSE =  2.754427
Epoch 100 MSE =  0.63222194
Epoch 200 MSE =  0.5727803
Epoch 300 MSE =  0.5585008
Epoch 400 MSE =  0.54907006
Epoch 500 MSE =  0.542288
Epoch 600 MSE =  0.5373791
Epoch 700 MSE =  0.533822
Epoch 800 MSE =  0.53124255
Epoch 900 MSE =  0.5293705


# Using Autodiff
- You could use symbolic differentiation to automatically find the equations for the partial derivatives.

In [12]:
reset_graph()

n_epochs = 1000
learning_rate = 0.1

X = tf.constant(scaled_housing_data_plus_bias, dtype = tf.float32, name = 'x')
y = tf.constant(housing.target.reshape(-1,1), dtype = tf.float32, name='y')
theta =  tf.Variable(tf.random.uniform([n+1,1], -1.0, 1.0, seed=42), name='theta')

y_pred = tf.matmul(X, theta, name = 'predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')

In [13]:
gradients = tf.gradients(mse,[theta])[0] # Autodiff

In [18]:
training_op = tf.compat.v1.assign(theta, theta - learning_rate * gradients)
init = tf.compat.v1.global_variables_initializer()

with tf.compat.v1.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print('Epoch', epoch, 'MSE = ', mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()

print('\nBest theta: ', best_theta)

Epoch 0 MSE =  2.754427
Epoch 100 MSE =  0.5279282
Epoch 200 MSE =  0.5244983
Epoch 300 MSE =  0.5243345
Epoch 400 MSE =  0.52432257
Epoch 500 MSE =  0.5243212
Epoch 600 MSE =  0.524321
Epoch 700 MSE =  0.52432096
Epoch 800 MSE =  0.524321
Epoch 900 MSE =  0.52432096

Best theta:  [[ 2.0685577 ]
 [ 0.8296135 ]
 [ 0.11875056]
 [-0.26551595]
 [ 0.30568725]
 [-0.00450334]
 [-0.03932605]
 [-0.89989907]
 [-0.8705536 ]]


In [19]:
def my_func(a,b):
    z = 0 
    for i in range(100):
        z = a * np.cos(z+i) + z * np.sin(b - i)
        return z

In [20]:
my_func(0.2,0.3)

0.2

In [23]:
reset_graph()

a = tf.Variable(0.2, name = 'a')
b = tf.Variable(0.3, name= 'b')
z = tf.constant(0.0, name= 'z0')
for i in range(100):
    z = a * tf.cos(z+i) + z * tf.sin(b - i)
    
grads = tf.compat.v1.gradients(z, [a,b])
init = tf.compat.v1.global_variables_initializer()

Let's compute the function at $a=0.2$ and $b=0.3$, and the partial derivatives at that point with regards to $a$ and with regards to $b$

In [24]:
with tf.compat.v1.Session() as sess:
    init.run()
    print(z.eval)
    print(sess.run(grads))

<bound method Tensor.eval of <tf.Tensor 'add_199:0' shape=() dtype=float32>>
[-1.1388495, 0.19671395]


# Using a GradientDescentOptimizer

In [25]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name = 'X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name = 'Y')
theta = tf.Variable(tf.random.uniform([n+1, 1], -1,1, seed=42), name = 'theta')

y_pred = tf.matmul(X, theta, name = 'predictions')
error = y_pred - y

mse = tf.reduce_mean(tf.square(error), name = 'mse')

In [27]:
optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

In [29]:
init = tf.compat.v1.global_variables_initializer()

with tf.compat.v1.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print('Epoch', epoch, 'MSE', mse.eval())
            
        sess.run(training_op)
        
    best_theta = theta.eval()
print('\nBest theta', best_theta)


Epoch 0 MSE 2.754427
Epoch 100 MSE 0.63222194
Epoch 200 MSE 0.5727803
Epoch 300 MSE 0.5585009
Epoch 400 MSE 0.54907006
Epoch 500 MSE 0.542288
Epoch 600 MSE 0.5373791
Epoch 700 MSE 0.533822
Epoch 800 MSE 0.53124255
Epoch 900 MSE 0.5293704

Best theta [[ 2.06855249e+00]
 [ 7.74078071e-01]
 [ 1.31192386e-01]
 [-1.17845066e-01]
 [ 1.64778143e-01]
 [ 7.44078017e-04]
 [-3.91945094e-02]
 [-8.61356676e-01]
 [-8.23479772e-01]]


# Using a Momentum Optimizer

In [32]:
reset_graph()

n_epochs = 1000
learning_rate = 0.1

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1),dtype=tf.float32, name='y')
theta = tf.Variable(tf.random.uniform([n+1,1], -1.0,1.0,seed=42), name='theta')

y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')

In [36]:
optimizer = tf.compat.v1.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)

In [37]:
training_op = optimizer.minimize(mse)
init = tf.compat.v1.global_variables_initializer()

In [38]:
with tf.compat.v1.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        sess.run(training_op)
    
    best_theta = theta.eval()

print("Best theta:", best_theta)

Best theta: [[ 2.0685582 ]
 [ 0.8296194 ]
 [ 0.1187517 ]
 [-0.26552704]
 [ 0.30569637]
 [-0.004503  ]
 [-0.03932628]
 [-0.8998852 ]
 [-0.87054056]]


# Feeding Data to the Training Algorithm

## PlaceHolder Nodes
- Replace X and y at every iteration with the next mini-batch and the best way is to use placeholder nodes.
- These nodes are special because they don’t actually perform any computation
- They just output the data you tell them to output at runtime.
- They are typically used to pass the training data to TensorFlow during training.

In [39]:
reset_graph()

A = tf.compat.v1.placeholder(tf.float32, shape=(None, 3))
B = A + 5

with tf.compat.v1.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1,2,3]]})  # Used to specify the value of A
    B_val_2 = B.eval(feed_dict={A: [[4,5,6], [7,8,9]]})
print(B_val_1)

[[6. 7. 8.]]


In [40]:
print(B_val_2)

[[ 9. 10. 11.]
 [12. 13. 14.]]


# Mini-Batch Gradient Descent

In [42]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.compat.v1.placeholder(tf.float32, shape=(None, n+1), name='X')
y = tf.compat.v1.placeholder(tf.float32,shape=(None, 1), name='Y')

In [43]:
theta = tf.Variable(tf.random.uniform([n+1,1], -1.0,1.0, seed=42), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y

mse = tf.reduce_mean(tf.square(error), name='mse')
optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.compat.v1.global_variables_initializer()

In [47]:
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m/batch_size))

def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)
    indices = np.random.randint(m, size = batch_size)
    
    X_batch = scaled_housing_data_plus_bias[indices]
    y_batch = housing.target.reshape(-1,1)[indices]
    
    return X_batch, y_batch

with tf.compat.v1.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
            best_theta = theta.eval()
print('Best_theta: ', best_theta)

Best_theta:  [[ 2.070016  ]
 [ 0.8204561 ]
 [ 0.1173173 ]
 [-0.22739051]
 [ 0.3113402 ]
 [ 0.00353193]
 [-0.01126994]
 [-0.91643935]
 [-0.8795008 ]]


# Saving and Restoring a Model

In [50]:
reset_graph()

n_epochs = 1000                                                                       
learning_rate = 0.01                                                                

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")            
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")          
theta = tf.Variable(tf.random.uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")

y_pred = tf.matmul(X, theta, name="predictions")                                     
error = y_pred - y                                                                  
mse = tf.reduce_mean(tf.square(error), name="mse") 

optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=learning_rate)            
training_op = optimizer.minimize(mse)    

init = tf.compat.v1.global_variables_initializer()
saver = tf.compat.v1.train.Saver()

with tf.compat.v1.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())                                
            save_path = saver.save(sess, "/Users/User/TensorFlow 1")
        sess.run(training_op)
    
    best_theta = theta.eval()
    save_path = saver.save(sess, "/Users/User/TensorFlow 1")

Epoch 0 MSE = 2.754427
Epoch 100 MSE = 0.63222194
Epoch 200 MSE = 0.5727803
Epoch 300 MSE = 0.5585009
Epoch 400 MSE = 0.54907006
Epoch 500 MSE = 0.542288
Epoch 600 MSE = 0.5373791
Epoch 700 MSE = 0.533822
Epoch 800 MSE = 0.53124255
Epoch 900 MSE = 0.5293704


In [51]:
best_theta

array([[ 2.06855249e+00],
       [ 7.74078071e-01],
       [ 1.31192386e-01],
       [-1.17845066e-01],
       [ 1.64778143e-01],
       [ 7.44078017e-04],
       [-3.91945094e-02],
       [-8.61356676e-01],
       [-8.23479772e-01]], dtype=float32)

In [53]:
with tf.compat.v1.Session() as sess:
    saver.restore(sess, "/Users/User/TensorFlow 1")
    best_theta_restored = theta.eval() 

INFO:tensorflow:Restoring parameters from /Users/User/TensorFlow 1


In [58]:
saver = tf.compat.v1.train.Saver({'weights': theta})

In [62]:
reset_graph()# notice that we start with an empty graph.

saver = tf.compat.v1.train.import_meta_graph("/Users/User/TensorFlow 1.meta")  # this loads the graph structure
theta = tf.compat.v1.get_default_graph().get_tensor_by_name("theta:0") 

with tf.compat.v1.Session() as sess:
    saver.restore(sess, "/Users/User/TensorFlow 1")  # this restores the graph's state
    best_theta_restored = theta.eval() 

INFO:tensorflow:Restoring parameters from /Users/User/TensorFlow 1


np.allclose(best_theta, best_theta_restored)

# Visualizing the Graph and Training Curves Using TensorBoard.

In [68]:
%load_ext tensorboard
from datetime import datetime
tf.compat.v1.disable_eager_execution()

reset_graph()

now = datetime.utcnow().strftime('%Y%m%d%H%M%S') # prevents different runs when working with tensorboard.
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)
# To reload tensorboard use - %reload_ext tensorboard.

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [70]:
n_epochs = 1000
learning_rate = 0.01

X = tf.compat.v1.placeholder(tf.float32, shape=(None, n + 1), name="X")# Used to pass data into Tensorflow.
y = tf.compat.v1.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random.uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")

y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.compat.v1.global_variables_initializer()


In [72]:
mse_summary = tf.summary.scalar('MSE', mse)
# node in the graph that will evaluate the MSE value and write it to a TensorBoard compatible binary log string.

file_writer = tf.compat.v1.summary.FileWriter(logdir, tf.compat.v1.get_default_graph())
#writes summaries to logfiles in the log directory.

In [82]:
# This code updates the execution phase to evaluate the mse_summary node regularly during training.
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

with tf.compat.v1.Session() as sess:                                                        
    sess.run(init)                                                                

    for epoch in range(n_epochs):                                                 
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.flush=(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()                                                     
file_writer.close() 

In [83]:
best_theta

array([[ 2.070016  ],
       [ 0.8204561 ],
       [ 0.1173173 ],
       [-0.22739051],
       [ 0.3113402 ],
       [ 0.00353193],
       [-0.01126994],
       [-0.91643935],
       [-0.8795008 ]], dtype=float32)

# Name Scopes

In [85]:
reset_graph()

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

n_epochs = 1000
learning_rate = 0.01

X = tf.compat.v1.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.compat.v1.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random.uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")

In [86]:
# you can create name scopes to group related nodes,
# NameSpaces helps avoid cluttering with thousands of nodes.

with tf.name_scope("loss") as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name="mse")


In [89]:
optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.compat.v1.global_variables_initializer()

mse_summary = tf.compat.v1.summary.scalar('MSE', mse)
file_writer = tf.compat.v1.summary.FileWriter(logdir, tf.compat.v1.get_default_graph())

In [90]:
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

with tf.compat.v1.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()

file_writer.flush()
file_writer.close()
print("Best theta:")
print(best_theta)

Best theta:
[[ 2.070016  ]
 [ 0.8204561 ]
 [ 0.1173173 ]
 [-0.22739051]
 [ 0.3113402 ]
 [ 0.00353193]
 [-0.01126994]
 [-0.91643935]
 [-0.8795008 ]]


In [91]:
print(error.op.name)

loss/sub


In [92]:
print(mse.op.name)

loss/mse


In [93]:
reset_graph()

a1 = tf.Variable(0, name="a")      # name == "a"
a2 = tf.Variable(0, name="a")      # name == "a_1"

with tf.name_scope("param"):       # name == "param"
    a3 = tf.Variable(0, name="a")  # name == "param/a"

with tf.name_scope("param"):       # name == "param_1"
    a4 = tf.Variable(0, name="a")  # name == "param_1/a"

for node in (a1, a2, a3, a4):
    print(node.op.name)

a
a_1
param/a
param_1/a


# Modularity

In [95]:
reset_graph()
# Recified Linear Unit:  A ReLU computes a linear function of the inputs, 
# and outputs the result if it is positive, and 0 otherwise.

n_features = 3
X = tf.compat.v1.placeholder(tf.float32, shape=(None, n_features), name="X")

w1 = tf.Variable(tf.random.normal((n_features, 1)), name="weights1")
w2 = tf.Variable(tf.random.normal((n_features, 1)), name="weights2")
b1 = tf.Variable(0.0, name="bias1")
b2 = tf.Variable(0.0, name="bias2")

z1 = tf.add(tf.matmul(X, w1), b1, name="z1")
z2 = tf.add(tf.matmul(X, w2), b2, name="z2")

relu1 = tf.maximum(z1, 0., name="relu1")
relu2 = tf.maximum(z1, 0., name="relu2")  # Oops, cut&paste error! Did you spot it?

output = tf.add(relu1, relu2, name="output")

In [96]:
# Build ReLu function
reset_graph()

def relu(X):
    w_shape = (int(X.get_shape()[1]), 1)
    w = tf.Variable(tf.random.normal(w_shape), name="weights")
    b = tf.Variable(0.0, name="bias")
    z = tf.add(tf.matmul(X, w), b, name="z")
    return tf.maximum(z, 0., name="relu")

n_features = 3
X = tf.compat.v1.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

In [98]:
file_writer = tf.compat.v1.summary.FileWriter("logs/relu1", tf.compat.v1.get_default_graph())

In [99]:
n_features = 3
X = tf.compat.v1.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

file_writer = tf.compat.v1.summary.FileWriter("logs/relu2", tf.compat.v1.get_default_graph())
file_writer.close()

# Sharing Variables
Sharing a threshold variable the classic way, by defining it outside of the relu() function then passing it as a parameter

In [100]:
reset_graph()
#If you want to share a variable between various components of your graph, 
#one simple option is to create it first, then pass it as a parameter to the functions that need it

def relu(X, threshold):
    with tf.name_scope("relu"):
        w_shape = (int(X.get_shape()[1]), 1)                        
        w = tf.Variable(tf.random.normal(w_shape), name="weights")  
        b = tf.Variable(0.0, name="bias")                           
        z = tf.add(tf.matmul(X, w), b, name="z")                
        return tf.maximum(z, threshold, name="max")

threshold = tf.Variable(0.0, name="threshold")
X = tf.compat.v1.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X, threshold) for i in range(5)]
output = tf.add_n(relus, name="output")

In [101]:
reset_graph()

#ReLU class using class variables to handle the shared parameter.
#Another option is to set the shared variable as an attribute of the relu() function upon the first call.

def relu(X):
    with tf.name_scope("relu"):
        if not hasattr(relu, "threshold"):
            relu.threshold = tf.Variable(0.0, name="threshold")
        w_shape = int(X.get_shape()[1]), 1                      
        w = tf.Variable(tf.random.normal(w_shape), name="weights") 
        b = tf.Variable(0.0, name="bias")                           
        z = tf.add(tf.matmul(X, w), b, name="z")                    
        return tf.maximum(z, relu.threshold, name="max")

In [102]:
X = tf.compat.v1.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

In [104]:
#The idea is to use the get_variable() function to
#create the shared variable if it does not exist yet, or reuse it if it already exists.

with tf.compat.v1.variable_scope("relu"):
    threshold = tf.compat.v1.get_variable("threshold", shape=(),
                                initializer=tf.constant_initializer(0.0))

In [ ]:
with tf.compat.v1.variable_scope("relu", reuse=True):
    threshold = tf.compat.v1.get_variable("threshold")

In [105]:
with tf.compat.v1.variable_scope("relu") as scope:
    scope.reuse_variables()
    threshold = tf.compat.v1.get_variable("threshold")

In [107]:
reset_graph()

def relu(X):
    with tf.compat.v1.variable_scope("relu", reuse=True):
        threshold = tf.compat.v1.get_variable("threshold")
        w_shape = int(X.get_shape()[1]), 1                          
        w = tf.Variable(tf.random.normal(w_shape), name="weights")  
        b = tf.Variable(0.0, name="bias")                           
        z = tf.add(tf.matmul(X, w), b, name="z")                    
        return tf.maximum(z, threshold, name="max")

X = tf.compat.v1.placeholder(tf.float32, shape=(None, n_features), name="X")
with tf.compat.v1.variable_scope("relu"):
    threshold = tf.compat.v1.get_variable("threshold", shape=(),
                                initializer=tf.constant_initializer(0.0))
relus = [relu(X) for relu_index in range(5)]
output = tf.add_n(relus, name="output")

In [108]:
file_writer = tf.compat.v1.summary.FileWriter("logs/relu6", tf.compat.v1.get_default_graph())
file_writer.close()

In [110]:
reset_graph()

def relu(X):
    with tf.compat.v1.variable_scope("relu"):
        threshold = tf.compat.v1.get_variable("threshold", shape=(), initializer=tf.constant_initializer(0.0))
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random.normal(w_shape), name="weights")
        b = tf.Variable(0.0, name="bias")
        z = tf.add(tf.matmul(X, w), b, name="z")
        return tf.maximum(z, threshold, name="max")

X = tf.compat.v1.placeholder(tf.float32, shape=(None, n_features), name="X")
with tf.compat.v1.variable_scope("", default_name="") as scope:
    first_relu = relu(X)     # create the shared variable
    scope.reuse_variables()  # then reuse it
    relus = [first_relu] + [relu(X) for i in range(4)]
output = tf.add_n(relus, name="output")

file_writer = tf.compat.v1.summary.FileWriter("logs/relu8", tf.compat.v1.get_default_graph())
file_writer.close()

In [ ]:
reset_graph()

def relu(X):
    threshold = tf.compat.v1.get_variable("threshold", shape=(),
                                initializer=tf.constant_initializer(0.0))
    w_shape = (int(X.get_shape()[1]), 1)                        
    w = tf.Variable(tf.random.normal(w_shape), name="weights")  
    b = tf.Variable(0.0, name="bias")                           
    z = tf.add(tf.matmul(X, w), b, name="z")                
    return tf.maximum(z, threshold, name="max")

X = tf.compat.v1.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = []
for relu_index in range(5):
    with tf.compat.v1.variable_scope("relu", reuse=(relu_index >= 1)) as scope:
        relus.append(relu(X))
output = tf.add_n(relus, name="output")

In [111]:
file_writer = tf.compat.v1.summary.FileWriter("logs/relu9", tf.compat.v1.get_default_graph())
file_writer.close()